# !DA CONTINUARE SOLO A NORMALIZZAZIONE COMPLETATA!

In [5]:
# deps
!pip install rapidfuzz
!pip install python-Levenshtein

### Rapidfuzz
`rapidfuzz` è una libreria Python per la **fuzzy matching**, simile a `fuzzywuzzy`, ma più veloce e più efficiente perché scritta in C++. Il **fuzzy matching** è una tecnica di matching approssimativo che confronta due stringhe per determinare quanto sono simili, anche se non sono esattamente uguali.

Il Fuzzy Matching utilizza metriche di similarità testuale come:

+ **Levenshtein Distance** (distanza di edit)
+ **Jaro-Winkler Similarity**
+ **Token-based Matching** (ignora l'ordine delle parole)


In [6]:
from rapidfuzz import fuzz

# Distanza Levenshtein
str1 = "Google Inc."
str2 = "Gooogle Inc."

similarity = fuzz.ratio(str1, str2)
print(f"Similarità: {similarity}%")

Similarità: 95.65217391304348%


In [7]:
similarity = fuzz.token_sort_ratio("International Business Machines", "Machines International Business")
print(f"Token Sort Ratio: {similarity}%")

Token Sort Ratio: 100.0%


In [8]:
#Trova la Migliore Corrispondenza in una Lista
from rapidfuzz import process

choices = ["Google Inc.", "Amazon LLC", "Microsoft Corp.", "Apple Ltd."]
query = "Gooogle"

best_match = process.extractOne(query, choices)
print(best_match)

('Google Inc.', 83.07692307692308, 0)


# **Costruzione della Ground Truth**

1. Selezionare coppie candidate per il matching
2. Selezionare coppie di aziende che non corrispondono
3. Bilanciare la distribuzione dei casi facili e difficili
4. Validare manualmente un sottoinsieme delle coppie
5. Salvare la ground-truth in un formato utilizzabile per il training

## Step 1: Selezionare Coppie Candidate per il Matching

Dobbiamo creare un insieme di coppie di aziende che potrebbero essere la stessa entità.
Utilizziamo una combinazione di blocking e similarità fuzzy su più attributi (le strategie di blocking non sono quelle definitive che useremo nella fase di record linkage)

### Blocking
- Matching su nomi
- Matching su indirizzo
- Matching su telefono
### Similarità
- Nomi simili, usando **Jaccard, Levenshtein, Jaro-Winkler**
- Sede operativa
- Partita iva o codici identificativi

In [ ]:
import pandas as pd
from itertools import combinations
from rapidfuzz import fuzz

AZIENDE_CSV = '../aziende_normalizzate.csv'
companies_df = pd.read_csv(AZIENDE_CSV)

candidate_pairs = []

for idx1, idx2 in combinations(companies_df.index, 2):
    company1 = companies_df.loc[idx1]
    company2 = companies_df.loc[idx2]

    # Blocking: aziende nella stessa città e con nome simile
    if company1["city"] == company2["city"] and fuzz.token_sort_ratio(company1["company_name"], company2["company_name"]) > 85:
        candidate_pairs.append((company1["company_id"], company2["company_id"]))




/tmp/ipykernel_292239/2279105695.py:6: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  companies_df = pd.read_csv(AZIENDE_CSV)
